<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# To Check Solar's ability

In [4]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [5]:
# Additional Contents : https://wikidocs.net/253106 

In [6]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()
file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [7]:
# read test_samples.csv file

import pandas as pd
import os

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers


now_topic = 'law'

prompts, answers = read_data(os.path.join(data_path, f'testset.csv'))

## 4. Prompt engineering

In [8]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    ---
    Please provide most correct answer.
    The answer format is as follow.

    ---
    Question: "Which of the following is most likely to be found in a library?
    (A) A basketball
    (B) A book
    (C) A painting
    (D) A computer
    (E) A refrigerator"
    Answer: (B) A book
    ---
    
    ---
    Question: {question}
    Answer: 
    ---

    '''

)
chain = prompt_template | llm

responses1 = []

for prompt in prompts:
    response = chain.invoke({"question": prompt})
    responses1.append(response.content)

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    Please provide most correct answer. Let's think step by step.
    
    The Answer format is as follow (do not explain) : 
    Answer: (number) the answer choice
    ---
    
    Question: {question}
    Answer :
    ---
        

    '''

)
chain = prompt_template | llm

responses2 = []

for prompt in prompts:
    response = chain.invoke({"question": prompt})
    responses2.append(response.content)

In [10]:
for i in responses1:
    print(i)
    print('-')


답변: (C) 60일
-
답: (C) 4

해당 문장에서 "재입학은 a회에 한하여 할 수 있다"는 문장을 통해, 재입학은 최대 a번까지 가능하다는 것을 알 수 있습니다. 또한, "다만 제 28조제4호에 의하여 제적된 자는 제적된 날부터 b년이 경과한 후 재입학 할 수 있다"는 문장을 통해, 제적된 후 b년이 지나야 재입학이 가능하다는 것을 알 수 있습니다. 따라서, a와 b의 값을 구하기 위해서는 해당 문장의 구체적인 내용을 파악해야 합니다.
-
답변: (B) 18학점

해설: 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 18학점 이상 취득한 때에 부전공을 이수한 것으로 인정받을 수 있습니다.
-
정답: (D) 로라 : 휴학기간 경과 후 3주가 지났으나 갑자기 큰 사고가 났다는 정당한 이유 때문에 복학하지 못하였음.
-
Answer: (C) 110명
-
정답: (D) 등급: C, 성적점: 2

이유: 
1980학년도 이전에는 성적 등급에 따른 성적점 시스템이 현재와 다르게 적용되었습니다. 따라서 (D) 등급: C, 성적점: 2는 1980학년도 이전에는 적용되지 않은 성적점입니다.
-
답변: (B) 문학사
-
답변: (C) 졸업 직전 학기에 있는 학생
-
이화여자대학교의 설립 정신은 (C) 기독교 정신입니다.
-
답변: (B) 서대문구
-
답변: (C) 20시간
-
답변: (D) 연계전공이라 한다.
-
답변: (C) 등록금이 과오납된 경우에는 금액을 반환할 수 없다.
-
Answer: (A) 전공기초과목과 전공과목
-
답변: (A) 학칙 개정과 대학평의원회는 무관하다.
-
답변: (D) 12학점
-
답변: (C) 2.00
-
Answer: (D) Personal matters
-
정답: (D) 3.75
-
답변: (B) 복수 전공 중
-
답변: (D) 졸업 요건 충족 못함
-
답변: (A) 필수/선택
-
답변: (D) 문학사 - 소비자학
-
답변: (C) 학점취득특별시험의 방법은 총장과 위원회가 함께 정한다.
-
Answer: (A

In [11]:
#answers = answers[::100]

## Check Accuracy

In [12]:
try : del [[wrongdf]]
except : pass
wrongdf = pd.DataFrame(columns=['questionNum', 'answer', '1st_pred'])

In [13]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [16]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, responses1)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":"-"}
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":None}
        
acc = cnt/len(answers)*100
print(f"acc: {acc}%")
print()
print("1st wrong:", wrong)
wrongdf.loc[len(wrongdf)] = {"questionNum":"-", "answer" : "-", "1st_pred":acc}

----------
답변: (C) 60일
idx: 0 | generated answer: C, answer: (D)
----------
답: (C) 4

해당 문장에서 "재입학은 a회에 한하여 할 수 있다"는 문장을 통해, 재입학은 최대 a번까지 가능하다는 것을 알 수 있습니다. 또한, "다만 제 28조제4호에 의하여 제적된 자는 제적된 날부터 b년이 경과한 후 재입학 할 수 있다"는 문장을 통해, 제적된 후 b년이 지나야 재입학이 가능하다는 것을 알 수 있습니다. 따라서, a와 b의 값을 구하기 위해서는 해당 문장의 구체적인 내용을 파악해야 합니다.
idx: 1 | generated answer: C, answer: (A)
----------
답변: (B) 18학점

해설: 학생이 소속 학과 또는 전공 이외의 전공 교과목을 총장이 정하는 바에 따라 18학점 이상 취득한 때에 부전공을 이수한 것으로 인정받을 수 있습니다.
idx: 2 | generated answer: B, answer: (C)
----------
정답: (D) 로라 : 휴학기간 경과 후 3주가 지났으나 갑자기 큰 사고가 났다는 정당한 이유 때문에 복학하지 못하였음.
idx: 3 | generated answer: D, answer: (D)
----------
Answer: (C) 110명
idx: 4 | generated answer: C, answer: (C)
----------
정답: (D) 등급: C, 성적점: 2

이유: 
1980학년도 이전에는 성적 등급에 따른 성적점 시스템이 현재와 다르게 적용되었습니다. 따라서 (D) 등급: C, 성적점: 2는 1980학년도 이전에는 적용되지 않은 성적점입니다.
idx: 5 | generated answer: C, answer: (B)
----------
답변: (B) 문학사
idx: 6 | generated answer: B, answer: (D)
----------
답변: (C) 졸업 직전 학기에 있는 학생
idx: 7 | generated 

In [15]:
wrongdf.to_csv(results_path+f"solarmini_zeroshot_testset.csv", index=False)